In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [7]:
df = load_iris() 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.data, df.target, test_size=0.3, random_state=0)

# Pipeline creation
1. Standard Scaler - Data Preprocessing
2. Reduce Dimension - PCA
3. Apply classifier

## Why Pipelines..

1. It will allow us to cross validate a process rather than a model.
2. Like hyperparameter tuning we can hyperparameterize the preprocessing steps ie., using hyperparameter for tuning a grid with imputing missing values with different stragies

# Type 1

In [14]:
pipelines = []

In [15]:
pipelines.append(
    Pipeline([
        ('scaler1', StandardScaler()), # Data preprocessing, both Fit & Transform get applied
        ('pca1', PCA(n_components=2)), # Data preprocessing, both Fit & Transform get applied
        ('l_r', LogisticRegression(random_state = 0)) # Model algorithm, Only Fit get applied
    ])
)

In [16]:
pipelines.append(
    Pipeline([
        ('scaler2', StandardScaler()), # Data preprocessing, both Fit & Transform get applied
        ('pca2', PCA(n_components=2)), # Data preprocessing, both Fit & Transform get applied
        ('dtc', DecisionTreeClassifier(random_state = 0)) # Model algorithm, Only Fit get applied
    ])
)

In [17]:
pipelines.append(
    Pipeline([
        ('scaler3', StandardScaler()), # Data preprocessing, both Fit & Transform get applied
        ('pca3', PCA(n_components=2)), # Data preprocessing, both Fit & Transform get applied
        ('rfc', RandomForestClassifier(random_state = 0)) # Model algorithm, Only Fit get applied
    ])
)

In [19]:
best_accuracy = 0.0
best_classifier = 0.0
best_pipeline = ""

In [20]:
pipeline_dict = { 
    0: 'LogisticRegression', 
    1: 'DecisionTreeClassifier', 
    2: 'RandomForestClassifier'
}

for pl in pipelines:
  pl.fit(X_train, y_train)

In [22]:
for index, model in enumerate(pipelines):
  print("'{}', test accuracy: {}".format(pipeline_dict[index], model.score(X_test, y_test)))

'LogisticRegression', test accuracy: 0.8666666666666667
'DecisionTreeClassifier', test accuracy: 0.9111111111111111
'RandomForestClassifier', test accuracy: 0.9111111111111111


In [37]:
for index, model in enumerate(pipelines):
  model_score = model.score(X_test, y_test)
  best_accuracy = 0 if index == 0 else best_accuracy 
  if model_score > best_accuracy:
    best_accuracy = model.score(X_test, y_test)
    best_pipeline = model
    best_classifier = pipeline_dict[index]

print("Best performer", best_classifier, "with score:", best_accuracy)

Best performer DecisionTreeClassifier with score: 0.9111111111111111


# Type 2

In [39]:
import pandas as pd

In [43]:
titanic_df = pd.read_csv('http://bit.ly/kaggletrain')

In [44]:
titanic_df.shape

(891, 12)

In [45]:
titanic_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [48]:
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [49]:
titanic_df.duplicated().sum()

0

In [55]:
titanic_df = titanic_df.loc[titanic_df['Embarked'].notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

In [56]:
titanic_df.shape

(889, 4)

In [57]:
titanic_df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [59]:
X = titanic_df.loc[:, ['Pclass']]
y = titanic_df['Survived']

In [61]:
print("X {}, y {}".format(X.shape, y.shape))

X (889, 1), y (889,)


In [63]:
lr_1 = LogisticRegression(solver='lbfgs')

In [64]:
from sklearn.model_selection import cross_val_score

In [66]:
cross_val_score(lr_1, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

In [67]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [68]:
from sklearn.preprocessing import OneHotEncoder

In [69]:
ohe = OneHotEncoder(sparse=False)

In [71]:
ohe.fit_transform(titanic_df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [72]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [73]:
ohe.fit_transform(titanic_df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [74]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [77]:
X = titanic_df.iloc[:, 1:]

In [78]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [79]:
from sklearn.compose import make_column_transformer

In [80]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough'
)

In [81]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [82]:
from sklearn.pipeline import make_pipeline

In [83]:
pipe_1 = make_pipeline(column_trans, lr_1)

In [86]:
cross_val_score(pipe_1, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [88]:
X_new = X.sample(5, random_state=99)

In [89]:
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [91]:
pipe_1.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [92]:
pipe_1.predict(X_new) # Preprocessing on X_new, fit

array([1, 0, 1, 1, 0])